In [3]:
%matplotlib notebook

import os 
import glob 
import json

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt 
 
sns.set_context('notebook')
sns.set_style('whitegrid')
sns.set_palette('Set1')

In [4]:
def load_results(path="../data"):
    for host in os.listdir(path):
        for name in glob.glob(os.path.join(path, host, "metrics-*.json")):
            with open(name, 'r') as f:
                for line in f:
                    row = json.loads(line.strip())
                    row['host'] = host 
                    yield row

In [6]:
r

{'bandit': {'counts': [4,
   1,
   3,
   5,
   1,
   4,
   2,
   0,
   0,
   0,
   3,
   3,
   1,
   2,
   3,
   3,
   3,
   4,
   2,
   3,
   4,
   3,
   1],
  'history': {'arms': [15,
    11,
    5,
    6,
    18,
    3,
    5,
    17,
    16,
    0,
    14,
    10,
    15,
    17,
    3,
    0,
    17,
    3,
    2,
    15,
    3,
    19,
    20,
    14,
    22,
    1,
    21,
    4,
    21,
    14,
    16,
    21,
    18,
    13,
    20,
    6,
    2,
    19,
    20,
    0,
    12,
    19,
    2,
    5,
    11,
    11,
    20,
    13,
    16,
    0,
    5,
    17,
    3,
    10,
    10],
   'rewards': [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0.9,
    0.9,
    0.9,
    0.45,
    1,
    1,
    1,
    0.95,
    0.95,
    0.5,
    0.5,
    1,
    0.5,
    0.5,
    0.5,
    0.5,
    0.95,
    0.5,
    0.5,
    1,
    0.95,
    0.9500000000000001,
    0.95,
    0.5,
    0.5,
    1,
    1,
    0.95,
    0.5,
    0.5,
    1,
    0.5,
    1,
    0